In [1]:
import yaml
import os

with open('data.yaml', 'r') as file:
    data = yaml.safe_load(file)

print("Train path:", os.path.exists(data['train']))
print("Validation path:", os.path.exists(data['val']))
print("Test path:", os.path.exists(data['test']))

def count_images(directory):
    supported_formats = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.tif')
    return len([name for name in os.listdir(directory) if name.lower().endswith(supported_formats)])

print(f" The number of images in training set: {count_images(data['train'])}")
print(f" The number of images in validation set: {count_images(data['val'])}")
print(f" The number of images in test set: {count_images(data['test'])}")


Train path: True
Validation path: True
Test path: True
 The number of images in training set: 2969
 The number of images in validation set: 251
 The number of images in test set: 116


In [2]:
from ultralytics import YOLO
from pathlib import Path

absolute_from_relative_path = Path('data.yaml').resolve()

model = YOLO('yolov8n.pt')  # Load a pretrained model (recommended for training)
results = model.train(data=absolute_from_relative_path, epochs=20, imgsz=480)


New https://pypi.org/project/ultralytics/8.2.31 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.227 🚀 Python-3.9.18 torch-1.13.1 CPU (Apple M1)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/Users/alessiacolumban/Desktop/2023-2024-projectone-ctai-AlessCol7/PillPoint/AI/PillPoint.v6i.yolov8/data.yaml, epochs=20, patience=50, batch=16, imgsz=480, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, a

Fontconfig warning: ignoring UTF-8: not a valid region tag
[W NNPACK.cpp:53] Could not initialize NNPACK! Reason: Unsupported hardware.


Freezing layer 'model.22.dfl.conv.weight'


train: Scanning /Users/alessiacolumban/Desktop/2023-2024-projectone-ctai-AlessCol7/PillPoint/AI/PillPoint.v6i.yolov8/train/labels.cache... 2969 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2969/2969 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 2673, len(boxes) = 18615. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



val: Scanning /Users/alessiacolumban/Desktop/2023-2024-projectone-ctai-AlessCol7/PillPoint/AI/PillPoint.v6i.yolov8/valid/labels.cache... 251 images, 0 backgrounds, 0 corrupt: 100%|██████████| 251/251 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 139, len(boxes) = 1348. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.


Plotting labels to runs/detect/train5/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000455, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 480 train, 480 val
Using 0 dataloader workers
Logging results to runs/detect/train5
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/20         0G      2.048      4.664      1.699        241        480:   2%|▏         | 3/186 [00:14<14:07,  4.63s/it]

In [ ]:
import cv2  # OpenCV library for computer vision tasks
from ultralytics import YOLO  # YOLO model for object detection
from PIL import Image, ImageTk  # PIL for image processing and integration with Tkinter
import tkinter as tk  # Tkinter for GUI
import socket  # Socket for network communication
import threading  # Threading for concurrent operations
import time  # Time for handling time-related operations

# Load the trained YOLOv8 model
try:
    model = YOLO('runs/detect/train/weights/best.pt')
except Exception as e:
    print(f"Error loading model: {e}")
    exit()

# Define the class names
class_names = ['FRI-am', 'FRI-pm', 'Hands', 'MON-am', 'MON-pm', 'Open', 'Pill', 'PillBox', 'SAT-am', 'SAT-pm', 'SUN-am', 'SUN-pm', 'THU-am', 'THU-pm', 'TUE-am', 'TUE-pm', 'WED-am', 'WED-pm']

# Define colors for each class visualization
class_colors = {
    'FRI-am': (0, 255, 255),
    'FRI-pm': (255, 255, 0),
    'Hands': (0, 255, 0),
    'MON-am': (255, 0, 255),
    'MON-pm': (0, 255, 255),
    'Open': (255, 255, 0),
    'Pill': (255, 0, 0),
    'PillBox': (0, 255, 255),
    'SAT-am': (255, 165, 0),
    'SAT-pm': (255, 69, 0),
    'SUN-am': (128, 0, 128),
    'SUN-pm': (75, 0, 130),
    'THU-am': (0, 255, 255),
    'THU-pm': (0, 128, 128),
    'TUE-am': (255, 0, 255),
    'TUE-pm': (128, 0, 0),
    'WED-am': (0, 0, 255),
    'WED-pm': (0, 128, 0)
}

# Server address and port for Raspberry Pi
server_address = ('192.168.168.167', 1441)  # Change this to your Raspberry Pi IP and port

# Global vars for use in methods/threads
client_socket = None
receive_thread = None
shutdown_flag = threading.Event()
last_notification_time = 0
notification_interval = 1
current_day_time = None

def setup_socket_client():
    global client_socket, receive_thread
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)  # Create a socket instance
    client_socket.connect(server_address)  # Connect to specified server
    print("Connected to server")

# Start a thread to handle incoming messages
    receive_thread = threading.Thread(target=receive_messages, args=(client_socket, shutdown_flag))
    receive_thread.start()

def receive_messages(sock, shutdown_flag):
    global current_day_time
    sock.settimeout(1)  # Set a timeout on the socket so we can check shutdown_flag.is_set in the loop, instead of blocking
    try:
        while not shutdown_flag.is_set():  # As long as ctrl+c is not pressed
            try:
                data = sock.recv(1024)  # Try to receive 1024 bytes of data (maximum amount; can be less)
                if not data:  # When no data is received, try again (and shutdown flag is checked again)
                    break
                current_day_time = data.decode()
                print("Received from server:", current_day_time)  # Print the received data, or do something with it
            except socket.timeout:  # When no data comes within timeout, try again
                continue
    except Exception as e:
        if not shutdown_flag.is_set():
            print(f"Connection error: {e}")
    finally:
        sock.close()

def request_date_time():
    global client_socket
    try:
        client_socket.sendall("Request date and time".encode())
    except Exception as e:
        print(f"Failed to request date and time: {e}")

# Function to draw bounding boxes with different colors
def draw_bounding_boxes(image, results): #image, which is the image/frame to be annotated, and results, which is the detection results from the YOLO model.
    global last_notification_time, current_day_time #global to indicate that it will modify the global variables 
    boxes = results[0].boxes.xyxy.cpu().numpy()
    confs = results[0].boxes.conf.cpu().numpy()
    classes = results[0].boxes.cls.cpu().numpy() #It extracts the bounding box coordinates, confidence scores, and class indices from the detection results and converts them to numpy arrays for easier manipulation.
    
    pillbox_detected = False
    compartment_open_detected = False
    wrong_compartment_opened = False
    

    #Iterates over the detected boxes. For each box
    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = map(int, box) #Extracts the coordinates (x1, y1, x2, y2) and converts them to integers
        conf = confs[i]  #Retrieves the confidence score conf and class index cls
        cls = int(classes[i])  #Maps the class index to the corresponding class name label.
        label = class_names[cls]
        
        # Get color for the class
        color = class_colors.get(label, (0, 0, 255))  # Default to blue if not found
        
        # Draw bounding box and label
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        cv2.putText(image, f'{label} {confs[i]:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
        
        # Check if PillBox is detected
        if label == 'PillBox':
            pillbox_detected = True

        # Check if compartment is opened
        if label == 'Open':
            compartment_open_detected = True
            
        # Check if the compartment corresponds to the current day and time
        if pillbox_detected and compartment_open_detected:
            if current_day_time and label not in current_day_time:
                wrong_compartment_opened = True
            
        if wrong_compartment_opened:
            # Add logic to trigger buzzer notification
            send_buzzer_notification()

def send_buzzer_notification():
    global client_socket
    try:
        client_socket.sendall("Wrong pill detected".encode())
    except Exception as e:
        print(f"Failed to send buzzer notification: {e}")

# AI part
def process_frame(frame):
    # Perform inference with the YOLOv8 model
    try:
        results = model(frame)
    except Exception as e:
        print(f"Error performing inference: {e}")
        return frame

    # Annotate the frame with detection results
    draw_bounding_boxes(frame, results)

    return frame

# Tkinter GUI part
def update_frame():
    ret, frame = cap.read()
    if ret:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert colorspace from BGR to RGB
        frame = process_frame(frame) #Calls process_frame(frame) to perform object detection and annotation on the frame.
        img = Image.fromarray(frame)  #Converts the processed frame (which is a numpy array) to a PIL Image object using Image.fromarray.
        imgtk = ImageTk.PhotoImage(image=img) #Converts the PIL Image to a format suitable for displaying in a Tkinter GUI using ImageTk.PhotoImage.
        label_img.imgtk = imgtk #Updates the image displayed in the Tkinter Label widget (label_img) with the new frame by setting its image attribute to imgtk.
        label_img.configure(image=imgtk)
    else:
        print("Error reading frame from webcam")
    label_img.after(10, update_frame)  # Schedule the update after 10 milliseconds

# Initialize the webcam
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Setup Tkinter window
window = tk.Tk()
window.title("Pill Detection")
window.geometry("800x600")

# Create a label to display the webcam feed
label_img = tk.Label(window)
label_img.pack()

# Start the socket client
setup_socket_client()

# Request the initial date and time
request_date_time()

# Start the webcam feed update
update_frame()

# Start the Tkinter event loop
window.mainloop()

# Release the webcam
cap.release()


[ WARN:0@7.612] global /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_f6tvh9615u/croot/opencv-suite_1691620375715/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


Connected to server
Received from server: Tue-AM
Received from server: Tue-AM



[W NNPACK.cpp:53] Could not initialize NNPACK! Reason: Unsupported hardware.


0: 288x480 1 FRI-am, 1 MON-am, 1 Pill, 1 SAT-am, 1 SAT-pm, 1 SUN-am, 1 SUN-pm, 1 THU-am, 1 TUE-am, 1 TUE-pm, 1 WED-am, 2 WED-pms, 222.7ms
Speed: 7.3ms preprocess, 222.7ms inference, 11.2ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 2 MON-ams, 1 Pill, 1 PillBox, 1 SAT-am, 1 SAT-pm, 2 SUN-ams, 1 SUN-pm, 1 TUE-am, 1 TUE-pm, 1 WED-pm, 123.7ms
Speed: 1.1ms preprocess, 123.7ms inference, 0.6ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 2 MON-ams, 1 Pill, 1 PillBox, 1 SAT-am, 1 SAT-pm, 2 SUN-ams, 1 SUN-pm, 1 TUE-am, 1 TUE-pm, 1 WED-pm, 76.4ms
Speed: 1.1ms preprocess, 76.4ms inference, 0.6ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 1 Pill, 1 PillBox, 1 SAT-am, 1 SAT-pm, 2 SUN-ams, 1 SUN-pm, 2 TUE-pms, 1 WED-pm, 130.0ms
Speed: 1.3ms preprocess, 130.0ms inference, 0.7ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 2 MON-ams, 1 Pill, 1 PillBox, 1 SAT-am, 2 SUN-ams, 1 SUN-pm, 1 TUE-am, 1 TUE-pm, 1 WED-pm, 99.8ms
Speed: 5.7ms prepr

Received from server: Tue-AM
Received from server: Tue-AM


In [ ]:
import cv2  # OpenCV library for computer vision tasks
from ultralytics import YOLO  # YOLO model for object detection
from PIL import Image, ImageTk  # PIL for image processing and integration with Tkinter
import tkinter as tk  # Tkinter for GUI
import socket  # Socket for network communication
import threading  # Threading for concurrent operations
import time  # Time for handling time-related operations

# Load the trained YOLOv8 model
try:
    model = YOLO('runs/detect/train/weights/best.pt')
except Exception as e:
    print(f"Error loading model: {e}")
    exit()

# Define the class names
class_names = ['FRI-am', 'FRI-pm', 'Hands', 'MON-am', 'MON-pm', 'Open', 'Pill', 'PillBox', 'SAT-am', 'SAT-pm', 'SUN-am', 'SUN-pm', 'THU-am', 'THU-pm', 'TUE-am', 'TUE-pm', 'WED-am', 'WED-pm']

# Define colors for each class visualization
class_colors = {
    'FRI-am': (0, 255, 255),
    'FRI-pm': (255, 255, 0),
    'Hands': (0, 255, 0),
    'MON-am': (255, 0, 255),
    'MON-pm': (0, 255, 255),
    'Open': (255, 255, 0),
    'Pill': (255, 0, 0),
    'PillBox': (0, 255, 255),
    'SAT-am': (255, 165, 0),
    'SAT-pm': (255, 69, 0),
    'SUN-am': (128, 0, 128),
    'SUN-pm': (75, 0, 130),
    'THU-am': (0, 255, 255),
    'THU-pm': (0, 128, 128),
    'TUE-am': (255, 0, 255),
    'TUE-pm': (128, 0, 0),
    'WED-am': (0, 0, 255),
    'WED-pm': (0, 128, 0)
}

# Server address and port for Raspberry Pi
server_address = ('192.168.168.167', 1441)  # Change this to your Raspberry Pi IP and port

# Global vars for use in methods/threads
client_socket = None
receive_thread = None
shutdown_flag = threading.Event()
last_notification_time = 0
notification_interval = 1
current_day_time = None

def setup_socket_client():
    global client_socket, receive_thread
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)  # Create a socket instance
    client_socket.connect(server_address)  # Connect to specified server
    print("Connected to server")

# Start a thread to handle incoming messages
    receive_thread = threading.Thread(target=receive_messages, args=(client_socket, shutdown_flag))
    receive_thread.start()

def receive_messages(sock, shutdown_flag):
    global current_day_time
    sock.settimeout(1)  # Set a timeout on the socket so we can check shutdown_flag.is_set in the loop, instead of blocking
    try:
        while not shutdown_flag.is_set():  # As long as ctrl+c is not pressed
            try:
                data = sock.recv(1024)  # Try to receive 1024 bytes of data (maximum amount; can be less)
                if not data:  # When no data is received, try again (and shutdown flag is checked again)
                    break
                current_day_time = data.decode()
                print("Received from server:", current_day_time)  # Print the received data, or do something with it
            except socket.timeout:  # When no data comes within timeout, try again
                continue
    except Exception as e:
        if not shutdown_flag.is_set():
            print(f"Connection error: {e}")
    finally:
        sock.close()

def request_date_time():
    global client_socket
    try:
        client_socket.sendall("Request date and time".encode())
    except Exception as e:
        print(f"Failed to request date and time: {e}")

# Function to draw bounding boxes with different colors
def draw_bounding_boxes(image, results):
    global last_notification_time, current_day_time
    boxes = results[0].boxes.xyxy.cpu().numpy()
    confs = results[0].boxes.conf.cpu().numpy()
    classes = results[0].boxes.cls.cpu().numpy()
    
    pillbox_detected = False
    compartment_open_detected = False
    wrong_compartment_opened = False

    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = map(int, box)
        conf = confs[i]
        cls = int(classes[i])
        label = class_names[cls]
        
        color = class_colors.get(label, (0, 0, 255))  # Default to blue if not found
        
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        cv2.putText(image, f'{label} {confs[i]:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
        
        if label == 'PillBox':
            pillbox_detected = True

        if label == 'Open':
            compartment_open_detected = True
        
        if pillbox_detected and compartment_open_detected:
            if current_day_time and label not in current_day_time:
                wrong_compartment_opened = True

        # Debug statements to verify logic
        print(f"Label: {label}")
        print(f"Current Day and Time: {current_day_time}")
        print(f"Pillbox Detected: {pillbox_detected}")
        print(f"Compartment Open Detected: {compartment_open_detected}")
        print(f"Wrong Compartment Opened: {wrong_compartment_opened}")

    if wrong_compartment_opened:
        send_buzzer_notification()

            
        

def send_buzzer_notification():
    global client_socket
    try:
        client_socket.sendall("Wrong pill detected".encode())
    except Exception as e:
        print(f"Failed to send buzzer notification: {e}")

# AI part
def process_frame(frame):
    # Perform inference with the YOLOv8 model
    try:
        results = model(frame)
    except Exception as e:
        print(f"Error performing inference: {e}")
        return frame

    # Annotate the frame with detection results
    draw_bounding_boxes(frame, results)

    return frame

# Tkinter GUI part
def update_frame():
    ret, frame = cap.read()
    if ret:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert colorspace from BGR to RGB
        frame = process_frame(frame) #Calls process_frame(frame) to perform object detection and annotation on the frame.
        img = Image.fromarray(frame)  #Converts the processed frame (which is a numpy array) to a PIL Image object using Image.fromarray.
        imgtk = ImageTk.PhotoImage(image=img) #Converts the PIL Image to a format suitable for displaying in a Tkinter GUI using ImageTk.PhotoImage.
        label_img.imgtk = imgtk #Updates the image displayed in the Tkinter Label widget (label_img) with the new frame by setting its image attribute to imgtk.
        label_img.configure(image=imgtk)
    else:
        print("Error reading frame from webcam")
    label_img.after(10, update_frame)  # Schedule the update after 10 milliseconds

# Initialize the webcam
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Setup Tkinter window
window = tk.Tk()
window.title("Pill Detection")
window.geometry("800x600")

# Create a label to display the webcam feed
label_img = tk.Label(window)
label_img.pack()

# Start the socket client
setup_socket_client()

# Request the initial date and time
request_date_time()

# Start the webcam feed update
update_frame()

# Start the Tkinter event loop
window.mainloop()

# Release the webcam
cap.release()


[ WARN:0@7.758] global /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_f6tvh9615u/croot/opencv-suite_1691620375715/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


Connected to server
Received from server: Tue-PMTue-PM

0: 288x480 1 FRI-am, 1 FRI-pm, 1 MON-am, 1 MON-pm, 1 PillBox, 1 SAT-am, 1 SAT-pm, 1 SUN-am, 1 SUN-pm, 1 THU-am, 1 THU-pm, 1 TUE-am, 1 TUE-pm, 1 WED-am, 1 WED-pm, 84.2ms
Speed: 8.4ms preprocess, 84.2ms inference, 15.9ms postprocess per image at shape (1, 3, 288, 480)
Label: WED-pm
Current Day and Time: Tue-PMTue-PM
Pillbox Detected: False
Compartment Open Detected: False
Wrong Compartment Opened: False
Label: TUE-am
Current Day and Time: Tue-PMTue-PM
Pillbox Detected: False
Compartment Open Detected: False
Wrong Compartment Opened: False
Label: MON-am
Current Day and Time: Tue-PMTue-PM
Pillbox Detected: False
Compartment Open Detected: False
Wrong Compartment Opened: False
Label: TUE-pm
Current Day and Time: Tue-PMTue-PM
Pillbox Detected: False
Compartment Open Detected: False
Wrong Compartment Opened: False
Label: SAT-am
Current Day and Time: Tue-PMTue-PM
Pillbox Detected: False
Compartment Open Detected: False
Wrong Compartment O

[W NNPACK.cpp:53] Could not initialize NNPACK! Reason: Unsupported hardware.



0: 288x480 1 FRI-am, 1 FRI-pm, 1 MON-am, 1 MON-pm, 1 PillBox, 1 SAT-am, 1 SAT-pm, 1 SUN-am, 1 SUN-pm, 1 THU-am, 1 THU-pm, 1 TUE-am, 1 TUE-pm, 1 WED-am, 1 WED-pm, 56.6ms
Speed: 5.2ms preprocess, 56.6ms inference, 0.5ms postprocess per image at shape (1, 3, 288, 480)
Label: WED-pm
Current Day and Time: Tue-PMTue-PM
Pillbox Detected: False
Compartment Open Detected: False
Wrong Compartment Opened: False
Label: TUE-am
Current Day and Time: Tue-PMTue-PM
Pillbox Detected: False
Compartment Open Detected: False
Wrong Compartment Opened: False
Label: MON-am
Current Day and Time: Tue-PMTue-PM
Pillbox Detected: False
Compartment Open Detected: False
Wrong Compartment Opened: False
Label: WED-am
Current Day and Time: Tue-PMTue-PM
Pillbox Detected: False
Compartment Open Detected: False
Wrong Compartment Opened: False
Label: THU-am
Current Day and Time: Tue-PMTue-PM
Pillbox Detected: False
Compartment Open Detected: False
Wrong Compartment Opened: False
Label: TUE-pm
Current Day and Time: Tue-PMT

In [1]:
import cv2  # OpenCV library for computer vision tasks
from ultralytics import YOLO  # YOLO model for object detection
from PIL import Image, ImageTk  # PIL for image processing and integration with Tkinter
import tkinter as tk  # Tkinter for GUI
import socket  # Socket for network communication
import threading  # Threading for concurrent operations
import time  # Time for handling time-related operations

# Load the trained YOLOv8 model
try:
    model = YOLO('runs/detect/train3/weights/best.pt')
except Exception as e:
    print(f"Error loading model: {e}")
    exit()

# Define the class names
class_names = ['FRI-am', 'FRI-pm', 'Hands', 'MON-am', 'MON-pm', 'Open', 'Pill', 'PillBox', 'SAT-am', 'SAT-pm', 'SUN-am', 'SUN-pm', 'THU-am', 'THU-pm', 'TUE-am', 'TUE-pm', 'WED-am', 'WED-pm']

# Define colors for each class visualization
class_colors = {
    'FRI-am': (0, 255, 255),
    'FRI-pm': (255, 255, 0),
    'Hands': (0, 255, 0),
    'MON-am': (255, 0, 255),
    'MON-pm': (0, 255, 255),
    'Open': (255, 255, 0),
    'Pill': (255, 0, 0),
    'PillBox': (0, 255, 255),
    'SAT-am': (255, 165, 0),
    'SAT-pm': (255, 69, 0),
    'SUN-am': (128, 0, 128),
    'SUN-pm': (75, 0, 130),
    'THU-am': (0, 255, 255),
    'THU-pm': (0, 128, 128),
    'TUE-am': (255, 0, 255),
    'TUE-pm': (128, 0, 0),
    'WED-am': (0, 0, 255),
    'WED-pm': (0, 128, 0)
}

# Server address and port for Raspberry Pi
server_address = ('192.168.168.167', 1443)  # Change this to your Raspberry Pi IP and port

# Global vars for use in methods/threads
client_socket = None
receive_thread = None
shutdown_flag = threading.Event()
last_notification_time = 0
notification_interval = 1
current_day_time = None

def setup_socket_client():
    global client_socket, receive_thread
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)  # Create a socket instance
    client_socket.connect(server_address)  # Connect to specified server
    print("Connected to server")

    # Start a thread to handle incoming messages
    receive_thread = threading.Thread(target=receive_messages, args=(client_socket, shutdown_flag))
    receive_thread.start()

def receive_messages(sock, shutdown_flag):
    global current_day_time
    sock.settimeout(1)  # Set a timeout on the socket so we can check shutdown_flag.is_set in the loop, instead of blocking
    try:
        while not shutdown_flag.is_set():  # As long as ctrl+c is not pressed
            try:
                data = sock.recv(1024)  # Try to receive 1024 bytes of data (maximum amount; can be less)
                if not data:  # When no data is received, try again (and shutdown flag is checked again)
                    break
                current_day_time = data.decode()
                print("Received from server:", current_day_time)  # Print the received data, or do something with it
            except socket.timeout:  # When no data comes within timeout, try again
                continue
    except Exception as e:
        if not shutdown_flag.is_set():
            print(f"Connection error: {e}")
    finally:
        sock.close()

def request_date_time():
    global client_socket
    try:
        client_socket.sendall("Request date and time".encode())
    except Exception as e:
        print(f"Failed to request date and time: {e}")

# Function to draw bounding boxes with different colors
def draw_bounding_boxes(image, results):
    global last_notification_time, current_day_time
    boxes = results[0].boxes.xyxy.cpu().numpy()
    confs = results[0].boxes.conf.cpu().numpy()
    classes = results[0].boxes.cls.cpu().numpy()
    
    pillbox_detected = False
    compartment_open_detected = False
    wrong_compartment_opened = False

    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = map(int, box)
        conf = confs[i]
        cls = int(classes[i])
        label = class_names[cls]
        
        color = class_colors.get(label, (0, 0, 255))  # Default to blue if not found
        
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        cv2.putText(image, f'{label} {confs[i]:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
        
        if label == 'PillBox':
            pillbox_detected = True

        if label == 'Open':
            compartment_open_detected = True
            
            # Check if the open compartment matches the expected compartment
            if current_day_time and label not in current_day_time:
                wrong_compartment_opened = True

        # Debug statements to verify logic
        print(f"Label: {label}")
        print(f"Current Day and Time: {current_day_time}")
        print(f"Pillbox Detected: {pillbox_detected}")
        print(f"Compartment Open Detected: {compartment_open_detected}")
        print(f"Wrong Compartment Opened: {wrong_compartment_opened}")

    if pillbox_detected and compartment_open_detected and wrong_compartment_opened:
        send_buzzer_notification()

def send_buzzer_notification():
    global client_socket
    try:
        client_socket.sendall("Wrong pill detected".encode())
    except Exception as e:
        print(f"Failed to send buzzer notification: {e}")

# AI part
def process_frame(frame):
    # Perform inference with the YOLOv8 model
    try:
        results = model(frame)
    except Exception as e:
        print(f"Error performing inference: {e}")
        return frame

    # Annotate the frame with detection results
    draw_bounding_boxes(frame, results)

    return frame

# Tkinter GUI part
def update_frame():
    ret, frame = cap.read()
    if ret:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert colorspace from BGR to RGB
        frame = process_frame(frame) #Calls process_frame(frame) to perform object detection and annotation on the frame.
        img = Image.fromarray(frame)  #Converts the processed frame (which is a numpy array) to a PIL Image object using Image.fromarray.
        imgtk = ImageTk.PhotoImage(image=img) #Converts the PIL Image to a format suitable for displaying in a Tkinter GUI using ImageTk.PhotoImage.
        label_img.imgtk = imgtk #Updates the image displayed in the Tkinter Label widget (label_img) with the new frame by setting its image attribute to imgtk.
        label_img.configure(image=imgtk)
    else:
        print("Error reading frame from webcam")
    label_img.after(10, update_frame)  # Schedule the update after 10 milliseconds

# Initialize the webcam
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Setup Tkinter window
window = tk.Tk()
window.title("Pill Detection")
window.geometry("800x600")

# Create a label to display the webcam feed
label_img = tk.Label(window)
label_img.pack()

# Start the socket client
setup_socket_client()

# Request the initial date and time
request_date_time()

# Start the webcam feed update
update_frame()

# Start the Tkinter event loop
window.mainloop()

# Release the webcam
cap.release()


[ WARN:0@1.353] global /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_f6tvh9615u/croot/opencv-suite_1691620375715/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


Connected to server
Received from server: Tue-PMTue-PM

0: 288x480 1 MON-pm, 1 Open, 1 WED-am, 69.1ms
Speed: 4.1ms preprocess, 69.1ms inference, 1.1ms postprocess per image at shape (1, 3, 288, 480)
Label: Open
Current Day and Time: Tue-PMTue-PM
Pillbox Detected: False
Compartment Open Detected: True
Wrong Compartment Opened: True
Label: WED-am
Current Day and Time: Tue-PMTue-PM
Pillbox Detected: False
Compartment Open Detected: True
Wrong Compartment Opened: True
Label: MON-pm
Current Day and Time: Tue-PMTue-PM
Pillbox Detected: False
Compartment Open Detected: True
Wrong Compartment Opened: True


[W NNPACK.cpp:53] Could not initialize NNPACK! Reason: Unsupported hardware.



0: 288x480 1 MON-pm, 1 Open, 51.6ms
Speed: 1.2ms preprocess, 51.6ms inference, 0.6ms postprocess per image at shape (1, 3, 288, 480)
Label: Open
Current Day and Time: Tue-PMTue-PM
Pillbox Detected: False
Compartment Open Detected: True
Wrong Compartment Opened: True
Label: MON-pm
Current Day and Time: Tue-PMTue-PM
Pillbox Detected: False
Compartment Open Detected: True
Wrong Compartment Opened: True

0: 288x480 1 Open, 1 WED-am, 49.2ms
Speed: 1.8ms preprocess, 49.2ms inference, 0.5ms postprocess per image at shape (1, 3, 288, 480)
Label: Open
Current Day and Time: Tue-PMTue-PM
Pillbox Detected: False
Compartment Open Detected: True
Wrong Compartment Opened: True
Label: WED-am
Current Day and Time: Tue-PMTue-PM
Pillbox Detected: False
Compartment Open Detected: True
Wrong Compartment Opened: True

0: 288x480 1 Open, 50.0ms
Speed: 1.1ms preprocess, 50.0ms inference, 0.5ms postprocess per image at shape (1, 3, 288, 480)
Label: Open
Current Day and Time: Tue-PMTue-PM
Pillbox Detected: Fal

In [1]:
import cv2  # OpenCV library for computer vision tasks
from ultralytics import YOLO  # YOLO model for object detection
from PIL import Image, ImageTk  # PIL for image processing and integration with Tkinter
import tkinter as tk  # Tkinter for GUI
import socket  # Socket for network communication
import threading  # Threading for concurrent operations
import time  # Time for handling time-related operations

# Load the trained YOLOv8 model
try:
    model = YOLO('runs/detect/train3/weights/best.pt')
except Exception as e:
    print(f"Error loading model: {e}")
    exit()

# Define the class names
class_names = ['FRI-am', 'FRI-pm', 'Hands', 'MON-am', 'MON-pm', 'Open', 'Pill', 'PillBox', 'SAT-am', 'SAT-pm', 'SUN-am', 'SUN-pm', 'THU-am', 'THU-pm', 'TUE-am', 'TUE-pm', 'WED-am', 'WED-pm']

# Define colors for each class visualization
class_colors = {
    'FRI-am': (0, 255, 255),
    'FRI-pm': (255, 255, 0),
    'Hands': (0, 255, 0),
    'MON-am': (255, 0, 255),
    'MON-pm': (0, 255, 255),
    'Open': (255, 255, 0),
    'Pill': (255, 0, 0),
    'PillBox': (0, 255, 255),
    'SAT-am': (255, 165, 0),
    'SAT-pm': (255, 69, 0),
    'SUN-am': (128, 0, 128),
    'SUN-pm': (75, 0, 130),
    'THU-am': (0, 255, 255),
    'THU-pm': (0, 128, 128),
    'TUE-am': (255, 0, 255),
    'TUE-pm': (128, 0, 0),
    'WED-am': (0, 0, 255),
    'WED-pm': (0, 128, 0)
}

# Server address and port for Raspberry Pi
server_address = ('192.168.168.167', 1443)  # Change this to your Raspberry Pi IP and port

# Global vars for use in methods/threads
client_socket = None
receive_thread = None
shutdown_flag = threading.Event()
last_notification_time = 0
notification_interval = 1
current_day_time = None

def setup_socket_client():
    global client_socket, receive_thread
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)  # Create a socket instance
    client_socket.connect(server_address)  # Connect to specified server
    print("Connected to server")

    # Start a thread to handle incoming messages
    receive_thread = threading.Thread(target=receive_messages, args=(client_socket, shutdown_flag))
    receive_thread.start()

def receive_messages(sock, shutdown_flag):
    global current_day_time
    sock.settimeout(1)  # Set a timeout on the socket so we can check shutdown_flag.is_set in the loop, instead of blocking
    try:
        while not shutdown_flag.is_set():  # As long as ctrl+c is not pressed
            try:
                data = sock.recv(1024)  # Try to receive 1024 bytes of data (maximum amount; can be less)
                if not data:  # When no data is received, try again (and shutdown flag is checked again)
                    break
                current_day_time = data.decode()
                print("Received from server:", current_day_time)  # Print the received data, or do something with it
            except socket.timeout:  # When no data comes within timeout, try again
                continue
    except Exception as e:
        if not shutdown_flag.is_set():
            print(f"Connection error: {e}")
    finally:
        sock.close()

def request_date_time():
    global client_socket
    try:
        client_socket.sendall("Request date and time".encode())
    except Exception as e:
        print(f"Failed to request date and time: {e}")

# Function to draw bounding boxes with different colors
def draw_bounding_boxes(image, results):
    global last_notification_time, current_day_time
    boxes = results[0].boxes.xyxy.cpu().numpy()
    confs = results[0].boxes.conf.cpu().numpy()
    classes = results[0].boxes.cls.cpu().numpy()
    
    pillbox_detected = False
    compartment_open_detected = False
    wrong_compartment_opened = False

    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = map(int, box)
        conf = confs[i]
        cls = int(classes[i])
        label = class_names[cls]
        
        color = class_colors.get(label, (0, 0, 255))  # Default to blue if not found
        
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        cv2.putText(image, f'{label} {confs[i]:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
        
        if label == 'PillBox':
            pillbox_detected = True

        if label == 'Open':
            compartment_open_detected = True
            
            # Check if the open compartment matches the expected compartment
            if current_day_time and label not in current_day_time:
                wrong_compartment_opened = True

        # Debug statements to verify logic
        print(f"Label: {label}")
        print(f"Current Day and Time: {current_day_time}")
        print(f"Pillbox Detected: {pillbox_detected}")
        print(f"Compartment Open Detected: {compartment_open_detected}")
        print(f"Wrong Compartment Opened: {wrong_compartment_opened}")

    if pillbox_detected and compartment_open_detected and wrong_compartment_opened:
        send_buzzer_notification()

def send_buzzer_notification():
    global client_socket
    try:
        client_socket.sendall("Wrong pill detected".encode())
    except Exception as e:
        print(f"Failed to send buzzer notification: {e}")

# AI part
def process_frame(frame):
    # Perform inference with the YOLOv8 model
    try:
        results = model(frame)
    except Exception as e:
        print(f"Error performing inference: {e}")
        return frame

    # Annotate the frame with detection results
    draw_bounding_boxes(frame, results)

    return frame

# Tkinter GUI part
def update_frame():
    ret, frame = cap.read()
    if ret:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert colorspace from BGR to RGB
        frame = process_frame(frame) #Calls process_frame(frame) to perform object detection and annotation on the frame.
        img = Image.fromarray(frame)  #Converts the processed frame (which is a numpy array) to a PIL Image object using Image.fromarray.
        imgtk = ImageTk.PhotoImage(image=img) #Converts the PIL Image to a format suitable for displaying in a Tkinter GUI using ImageTk.PhotoImage.
        label_img.imgtk = imgtk #Updates the image displayed in the Tkinter Label widget (label_img) with the new frame by setting its image attribute to imgtk.
        label_img.configure(image=imgtk)
    else:
        print("Error reading frame from webcam")
    label_img.after(10, update_frame)  # Schedule the update after 10 milliseconds

# Initialize the webcam
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Setup Tkinter window
window = tk.Tk()
window.title("Pill Detection")
window.geometry("800x600")

# Create a label to display the webcam feed
label_img = tk.Label(window)
label_img.pack()

# Start the socket client
setup_socket_client()

# Request the initial date and time
request_date_time()

# Start the webcam feed update
update_frame()

# Start the Tkinter event loop
window.mainloop()

# Release the webcam
cap.release()


[ WARN:0@7.690] global /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_f6tvh9615u/croot/opencv-suite_1691620375715/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


ConnectionRefusedError: [Errno 61] Connection refused

: 

In [1]:
import cv2  # OpenCV library for computer vision tasks
from ultralytics import YOLO  # YOLO model for object detection
from PIL import Image, ImageTk  # PIL for image processing and integration with Tkinter
import tkinter as tk  # Tkinter for GUI
import socket  # Socket for network communication
import threading  # Threading for concurrent operations
import time  # Time for handling time-related operations
import logging  # For logging configurations
import sys  # For redirecting standard output
import os  # For setting environment variables
import contextlib  # For suppressing print statements

# Suppress logging from specific libraries
logging.getLogger('ultralytics').setLevel(logging.ERROR)
logging.getLogger('torch').setLevel(logging.ERROR)
logging.getLogger('PIL').setLevel(logging.ERROR)

# Suppress OpenCV debug prints using environment variable
os.environ['OPENCV_LOG_LEVEL'] = 'ERROR'

# Context manager to suppress print statements
@contextlib.contextmanager
def suppress_stdout():
    with open(os.devnull, 'w') as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout

# Load the trained YOLOv8 model
try:
    with suppress_stdout():
        model = YOLO('runs/detect/train3/weights/best.pt')
except Exception as e:
    print(f"Error loading model: {e}")
    exit()

# Define the class names
class_names = ['FRI-am', 'FRI-pm', 'Hands', 'MON-am', 'MON-pm', 'Open', 'Pill', 'PillBox', 'SAT-am', 'SAT-pm', 'SUN-am', 'SUN-pm', 'THU-am', 'THU-pm', 'TUE-am', 'TUE-pm', 'WED-am', 'WED-pm']

# Define colors for each class visualization
class_colors = {
    'FRI-am': (0, 255, 255),
    'FRI-pm': (255, 255, 0),
    'Hands': (0, 255, 0),
    'MON-am': (255, 0, 255),
    'MON-pm': (0, 255, 255),
    'Open': (255, 255, 0),
    'Pill': (255, 0, 0),
    'PillBox': (0, 255, 255),
    'SAT-am': (255, 165, 0),
    'SAT-pm': (255, 69, 0),
    'SUN-am': (128, 0, 128),
    'SUN-pm': (75, 0, 130),
    'THU-am': (0, 255, 255),
    'THU-pm': (0, 128, 128),
    'TUE-am': (255, 0, 255),
    'TUE-pm': (128, 0, 0),
    'WED-am': (0, 0, 255),
    'WED-pm': (0, 128, 0)
}

# Server address and port for Raspberry Pi
server_address = ('192.168.168.167', 1443)  # Change this to your Raspberry Pi IP and port

# Global vars for use in methods/threads
client_socket = None
receive_thread = None
shutdown_flag = threading.Event()
last_debug_time = time.time()
debug_interval = 2
current_day_time = None

def setup_socket_client():
    global client_socket, receive_thread
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)  # Create a socket instance
    client_socket.connect(server_address)  # Connect to specified server
    print("Connected to server")

    # Start a thread to handle incoming messages
    receive_thread = threading.Thread(target=receive_messages, args=(client_socket, shutdown_flag))
    receive_thread.start()

def receive_messages(sock, shutdown_flag):
    global current_day_time
    sock.settimeout(1)  # Set a timeout on the socket so we can check shutdown_flag.is_set in the loop, instead of blocking
    try:
        while not shutdown_flag.is_set():  # As long as ctrl+c is not pressed
            try:
                data = sock.recv(1024)  # Try to receive 1024 bytes of data (maximum amount; can be less)
                if not data:  # When no data is received, try again (and shutdown flag is checked again)
                    break
                current_day_time = data.decode()
                print("Received from server:", current_day_time)  # Print the received data, or do something with it
            except socket.timeout:  # When no data comes within timeout, try again
                continue
    except Exception as e:
        if not shutdown_flag.is_set():
            print(f"Connection error: {e}")
    finally:
        sock.close()

def request_date_time():
    global client_socket
    try:
        client_socket.sendall("Request date and time".encode())
    except Exception as e:
        print(f"Failed to request date and time: {e}")

# Function to draw bounding boxes with different colors
def draw_bounding_boxes(image, results):
    global last_debug_time, current_day_time
    boxes = results[0].boxes.xyxy.cpu().numpy()
    confs = results[0].boxes.conf.cpu().numpy()
    classes = results[0].boxes.cls.cpu().numpy()
    
    pillbox_detected = False
    compartment_open_detected = False
    wrong_compartment_opened = False

    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = map(int, box)
        conf = confs[i]
        cls = int(classes[i])
        label = class_names[cls]
        
        color = class_colors.get(label, (0, 0, 255))  # Default to blue if not found
        
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        cv2.putText(image, f'{label} {confs[i]:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
        
        if label == 'PillBox':
            pillbox_detected = True

        if label == 'Open':
            compartment_open_detected = True
            
        # Check if the open compartment matches the expected compartment
        if current_day_time and label not in current_day_time:
            print(f"Expected: {current_day_time}, Detected: {label}")
            wrong_compartment_opened = True

        # Check if it's time to print debug statements
        if last_debug_time + debug_interval < time.time():
            # Debug statements to verify logic
            print(f"Label: {label}")
            print(f"Current Day and Time: {current_day_time}")
            print(f"Pillbox Detected: {pillbox_detected}")
            print(f"Compartment Open Detected: {compartment_open_detected}")
            print(f"Wrong Compartment Opened: {wrong_compartment_opened}")
            # Update the last debug time
            last_debug_time = time.time()
        
    if pillbox_detected and compartment_open_detected and wrong_compartment_opened:
        send_buzzer_notification()

def send_buzzer_notification():
    global client_socket
    try:
        client_socket.sendall("Wrong pill detected".encode())
    except Exception as e:
        print(f"Failed to send buzzer notification: {e}")

# AI part
def process_frame(frame):
    # Perform inference with the YOLOv8 model
    try:
        with suppress_stdout():
            results = model(frame)
    except Exception as e:
        print(f"Error performing inference: {e}")
        return frame

    # Annotate the frame with detection results
    draw_bounding_boxes(frame, results)

    return frame

# Tkinter GUI part
def update_frame():
    ret, frame = cap.read()
    if ret:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert colorspace from BGR to RGB
        frame = process_frame(frame)  # Calls process_frame(frame) to perform object detection and annotation on the frame.
        img = Image.fromarray(frame)  # Converts the processed frame (which is a numpy array) to a PIL Image object using Image.fromarray.
        imgtk = ImageTk.PhotoImage(image=img)  # Converts the PIL Image to a format suitable for displaying in a Tkinter GUI using ImageTk.PhotoImage.
        label_img.imgtk = imgtk  # Updates the image displayed in the Tkinter Label widget (label_img) with the new frame by setting its image attribute to imgtk.
        label_img.configure(image=imgtk)
    else:
        print("Error reading frame from webcam")
    label_img.after(10, update_frame)  # Schedule the update after 10 milliseconds

# Initialize the webcam
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Setup Tkinter window
window = tk.Tk()
window.title("Pill Detection")
window.geometry("800x600")

# Create a label to display the webcam feed
label_img = tk.Label(window)
label_img.pack()

# Start the socket client
setup_socket_client()

# Request the initial date and time
request_date_time()

# Start the webcam feed update
update_frame()

# Start the Tkinter event loop
window.mainloop()

# Release the webcam
cap.release()


[ WARN:0@7.310] global /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_f6tvh9615u/croot/opencv-suite_1691620375715/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


Connected to server
Received from server: Wed-PM
Received from server: Wed-PM


[W NNPACK.cpp:53] Could not initialize NNPACK! Reason: Unsupported hardware.


Label: SAT-am
Current Day and Time: Wed-PM
Pillbox Detected: False
Compartment Open Detected: False
Wrong Compartment Opened: True
Label: SAT-am
Current Day and Time: Wed-PM
Pillbox Detected: False
Compartment Open Detected: False
Wrong Compartment Opened: True
Label: SAT-am
Current Day and Time: Wed-PM
Pillbox Detected: False
Compartment Open Detected: False
Wrong Compartment Opened: True
Label: SAT-am
Current Day and Time: Wed-PM
Pillbox Detected: False
Compartment Open Detected: False
Wrong Compartment Opened: True
Label: SAT-am
Current Day and Time: Wed-PM
Pillbox Detected: False
Compartment Open Detected: False
Wrong Compartment Opened: True
Label: SAT-am
Current Day and Time: Wed-PM
Pillbox Detected: False
Compartment Open Detected: False
Wrong Compartment Opened: True
Label: SAT-am
Current Day and Time: Wed-PM
Pillbox Detected: False
Compartment Open Detected: False
Wrong Compartment Opened: True
Label: SAT-am
Current Day and Time: Wed-PM
Pillbox Detected: False
Compartment Open

In [1]:
import cv2  # OpenCV library for computer vision tasks
from ultralytics import YOLO  # YOLO model for object detection
from PIL import Image, ImageTk  # PIL for image processing and integration with Tkinter
import tkinter as tk  # Tkinter for GUI
import socket  # Socket for network communication
import threading  # Threading for concurrent operations
import time  # Time for handling time-related operations
import logging  # For logging configurations
import sys  # For redirecting standard output
import os  # For setting environment variables
import contextlib  # For suppressing print statements

# Suppress logging from specific libraries
logging.getLogger('ultralytics').setLevel(logging.ERROR)
logging.getLogger('torch').setLevel(logging.ERROR)
logging.getLogger('PIL').setLevel(logging.ERROR)

# Suppress OpenCV debug prints using environment variable
os.environ['OPENCV_LOG_LEVEL'] = 'ERROR'

# Context manager to suppress print statements
@contextlib.contextmanager
def suppress_stdout():
    with open(os.devnull, 'w') as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout

# Load the trained YOLOv8 model
try:
    with suppress_stdout():
        model = YOLO('runs/detect/train3/weights/best.pt')
except Exception as e:
    print(f"Error loading model: {e}")
    exit()

# Define the class names
class_names = ['FRI-am', 'FRI-pm', 'Hands', 'MON-am', 'MON-pm', 'Open', 'Pill', 'PillBox', 'SAT-am', 'SAT-pm', 'SUN-am', 'SUN-pm', 'THU-am', 'THU-pm', 'TUE-am', 'TUE-pm', 'WED-am', 'WED-pm']

# Define the day and time slots for the pillbox compartments
day_time_slots = ['MON-am', 'MON-pm', 'TUE-am', 'TUE-pm', 'WED-am', 'WED-pm', 'THU-am', 'THU-pm', 'FRI-am', 'FRI-pm', 'SAT-am', 'SAT-pm', 'SUN-am', 'SUN-pm']

# Define colors for each class visualization
class_colors = {
    'FRI-am': (0, 255, 255),
    'FRI-pm': (255, 255, 0),
    'Hands': (0, 255, 0),
    'MON-am': (255, 0, 255),
    'MON-pm': (0, 255, 255),
    'Open': (255, 255, 0),
    'Pill': (255, 0, 0),
    'PillBox': (0, 255, 255),
    'SAT-am': (255, 165, 0),
    'SAT-pm': (255, 69, 0),
    'SUN-am': (128, 0, 128),
    'SUN-pm': (75, 0, 130),
    'THU-am': (0, 255, 255),
    'THU-pm': (0, 128, 128),
    'TUE-am': (255, 0, 255),
    'TUE-pm': (128, 0, 0),
    'WED-am': (0, 0, 255),
    'WED-pm': (0, 128, 0)
}

# Server address and port for Raspberry Pi
server_address = ('192.168.168.167', 1443)  # Change this to your Raspberry Pi IP and port

# Global vars for use in methods/threads
client_socket = None
receive_thread = None
shutdown_flag = threading.Event()
last_debug_time = time.time()
debug_interval = 2
current_day_time = None

def setup_socket_client():
    global client_socket, receive_thread
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)  # Create a socket instance
    client_socket.connect(server_address)  # Connect to specified server
    print("Connected to server")

    # Start a thread to handle incoming messages
    receive_thread = threading.Thread(target=receive_messages, args=(client_socket, shutdown_flag))
    receive_thread.start()

def receive_messages(sock, shutdown_flag):
    global current_day_time
    sock.settimeout(1)  # Set a timeout on the socket so we can check shutdown_flag.is_set in the loop, instead of blocking
    try:
        while not shutdown_flag.is_set():  # As long as ctrl+c is not pressed
            try:
                data = sock.recv(1024)  # Try to receive 1024 bytes of data (maximum amount; can be less)
                if not data:  # When no data is received, try again (and shutdown flag is checked again)
                    break
                current_day_time = data.decode()
                print("Received from server:", current_day_time)  # Print the received data, or do something with it
            except socket.timeout:  # When no data comes within timeout, try again
                continue
    except Exception as e:
        if not shutdown_flag.is_set():
            print(f"Connection error: {e}")
    finally:
        sock.close()

def request_date_time():
    global client_socket
    try:
        client_socket.sendall("Request date and time".encode())
    except Exception as e:
        print(f"Failed to request date and time: {e}")

# Function to draw bounding boxes with different colors
def draw_bounding_boxes(image, results):
    global last_debug_time, current_day_time
    boxes = results[0].boxes.xyxy.cpu().numpy()
    confs = results[0].boxes.conf.cpu().numpy()
    classes = results[0].boxes.cls.cpu().numpy()
    
    pillbox_detected = False
    compartment_open_detected = False
    wrong_compartment_opened = False
    detected_labels = []  # Initialize an empty list to store detected labels

    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = map(int, box)
        conf = confs[i]
        cls = int(classes[i])
        label = class_names[cls]

        if label == 'Open':
            continue

        if label in day_time_slots:
            detected_labels.append(label)  # Append the detected label to the list

        color = class_colors.get(label, (0, 0, 255))  # Default to blue if not found
        
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        cv2.putText(image, f'{label} {confs[i]:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
        
        if label == 'PillBox':
            pillbox_detected = True

            
        # # Check if the open compartment matches the expected compartment
        # if label == ' Open' and current_day_time not in:
        #     print(f"Expected: {current_day_time}, Detected: {label}")
        #     wrong_compartment_opened = True

    # Check if it's time to print debug statements
    if last_debug_time + debug_interval < time.time():
        # Debug statements to verify logic
        print(f"Current Day and Time: {current_day_time}")
        print(f"Pillbox Detected: {pillbox_detected}")
        print(f"Compartment Open Detected: {compartment_open_detected}")
        print(f"Wrong Compartment Opened: {wrong_compartment_opened}")
        print(f"Detected Labels: {detected_labels}")
        # Update the last debug time
        last_debug_time = time.time()
        
    if pillbox_detected and compartment_open_detected and wrong_compartment_opened:
        send_buzzer_notification()

def send_buzzer_notification():
    global client_socket
    try:
        client_socket.sendall("Wrong pill detected".encode())
    except Exception as e:
        print(f"Failed to send buzzer notification: {e}")

# AI part
def process_frame(frame):
    # Perform inference with the YOLOv8 model
    try:
        with suppress_stdout():
            results = model(frame)
    except Exception as e:
        print(f"Error performing inference: {e}")
        return frame

    # Annotate the frame with detection results
    draw_bounding_boxes(frame, results)

    return frame

# Tkinter GUI part
def update_frame():
    ret, frame = cap.read()
    if ret:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert colorspace from BGR to RGB
        frame = process_frame(frame)  # Calls process_frame(frame) to perform object detection and annotation on the frame.
        img = Image.fromarray(frame)  # Converts the processed frame (which is a numpy array) to a PIL Image object using Image.fromarray.
        imgtk = ImageTk.PhotoImage(image=img)  # Converts the PIL Image to a format suitable for displaying in a Tkinter GUI using ImageTk.PhotoImage.
        label_img.imgtk = imgtk  # Updates the image displayed in the Tkinter Label widget (label_img) with the new frame by setting its image attribute to imgtk.
        label_img.configure(image=imgtk)
    else:
        print("Error reading frame from webcam")
    label_img.after(10, update_frame)  # Schedule the update after 10 milliseconds

# Initialize the webcam
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Setup Tkinter window
window = tk.Tk()
window.title("Pill Detection")
window.geometry("800x600")

# Create a label to display the webcam feed
label_img = tk.Label(window)
label_img.pack()

# Start the socket client
setup_socket_client()

# Request the initial date and time
request_date_time()

# Start the webcam feed update
update_frame()

# Start the Tkinter event loop
window.mainloop()

# Release the webcam
cap.release()


[ WARN:0@7.554] global /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_f6tvh9615u/croot/opencv-suite_1691620375715/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


Connected to server
Received from server: Wed-PM
Received from server: Wed-PM


[W NNPACK.cpp:53] Could not initialize NNPACK! Reason: Unsupported hardware.


Current Day and Time: Wed-PM
Pillbox Detected: False
Compartment Open Detected: False
Wrong Compartment Opened: False
Detected Labels: ['WED-am']
Current Day and Time: Wed-PM
Pillbox Detected: False
Compartment Open Detected: False
Wrong Compartment Opened: False
Detected Labels: ['WED-am']
Current Day and Time: Wed-PM
Pillbox Detected: False
Compartment Open Detected: False
Wrong Compartment Opened: False
Detected Labels: ['WED-am']
Current Day and Time: Wed-PM
Pillbox Detected: True
Compartment Open Detected: False
Wrong Compartment Opened: False
Detected Labels: ['TUE-pm', 'SUN-pm', 'TUE-am', 'WED-pm', 'WED-pm', 'SUN-am', 'MON-am', 'MON-pm', 'WED-am']
Current Day and Time: Wed-PM
Pillbox Detected: False
Compartment Open Detected: False
Wrong Compartment Opened: False
Detected Labels: ['WED-am', 'SAT-am', 'TUE-pm', 'WED-pm', 'SAT-pm', 'SUN-pm', 'THU-pm', 'THU-am', 'FRI-am', 'TUE-am', 'MON-pm', 'SUN-am', 'MON-am', 'FRI-pm']
Current Day and Time: Wed-PM
Pillbox Detected: False
Compartm

In [2]:
import cv2  # OpenCV library for computer vision tasks
from ultralytics import YOLO  # YOLO model for object detection
from PIL import Image, ImageTk  # PIL for image processing and integration with Tkinter
import tkinter as tk  # Tkinter for GUI
import socket  # Socket for network communication
import threading  # Threading for concurrent operations
import time  # Time for handling time-related operations

# Load the trained YOLOv8 model
try:
    model = YOLO('runs/detect/train3/weights/best.pt')
except Exception as e:
    print(f"Error loading model: {e}")
    exit()

# Define the class names
class_names = ['FRI-am', 'FRI-pm', 'Hands', 'MON-am', 'MON-pm', 'Open', 'Pill', 'PillBox', 'SAT-am', 'SAT-pm', 'SUN-am', 'SUN-pm', 'THU-am', 'THU-pm', 'TUE-am', 'TUE-pm', 'WED-am', 'WED-pm']

# Define colors for each class visualization
class_colors = {
    'FRI-am': (0, 255, 255),
    'FRI-pm': (255, 255, 0),
    'Hands': (0, 255, 0),
    'MON-am': (255, 0, 255),
    'MON-pm': (0, 255, 255),
    'Open': (255, 255, 0),
    'Pill': (255, 0, 0),
    'PillBox': (0, 255, 255),
    'SAT-am': (255, 165, 0),
    'SAT-pm': (255, 69, 0),
    'SUN-am': (128, 0, 128),
    'SUN-pm': (75, 0, 130),
    'THU-am': (0, 255, 255),
    'THU-pm': (0, 128, 128),
    'TUE-am': (255, 0, 255),
    'TUE-pm': (128, 0, 0),
    'WED-am': (0, 0, 255),
    'WED-pm': (0, 128, 0)
}

# Server address and port for Raspberry Pi
server_address = ('192.168.168.167', 1443)  # Change this to your Raspberry Pi IP and port

# Global vars for use in methods/threads
client_socket = None
receive_thread = None
shutdown_flag = threading.Event()
last_notification_time = 0
notification_interval = 1
current_day_time = None

def setup_socket_client():
    global client_socket, receive_thread
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)  # Create a socket instance
    client_socket.connect(server_address)  # Connect to specified server
    print("Connected to server")

    # Start a thread to handle incoming messages
    receive_thread = threading.Thread(target=receive_messages, args=(client_socket, shutdown_flag))
    receive_thread.start()

def receive_messages(sock, shutdown_flag):
    global current_day_time
    sock.settimeout(1)  # Set a timeout on the socket so we can check shutdown_flag.is_set in the loop, instead of blocking
    try:
        while not shutdown_flag.is_set():  # As long as ctrl+c is not pressed
            try:
                data = sock.recv(1024)  # Try to receive 1024 bytes of data (maximum amount; can be less)
                if not data:  # When no data is received, try again (and shutdown flag is checked again)
                    break
                current_day_time = data.decode()
                print("Received from server:", current_day_time)  # Print the received data, or do something with it
            except socket.timeout:  # When no data comes within timeout, try again
                continue
    except Exception as e:
        if not shutdown_flag.is_set():
            print(f"Connection error: {e}")
    finally:
        sock.close()

def request_date_time():
    global client_socket
    try:
        client_socket.sendall("Request date and time".encode())
    except Exception as e:
        print(f"Failed to request date and time: {e}")

# Function to draw bounding boxes with different colors
def draw_bounding_boxes(image, results):
    global last_notification_time, current_day_time
    boxes = results[0].boxes.xyxy.cpu().numpy()
    confs = results[0].boxes.conf.cpu().numpy()
    classes = results[0].boxes.cls.cpu().numpy()
    
    pillbox_detected = False
    compartment_open_detected = False
    wrong_compartment_opened = False

    for i, box in enumerate(boxes):
        x1, y1, x2, y2 = map(int, box)
        conf = confs[i]
        cls = int(classes[i])
        label = class_names[cls]
        
        color = class_colors.get(label, (0, 0, 255))  # Default to blue if not found
        
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        cv2.putText(image, f'{label} {confs[i]:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
        
        if label == 'PillBox':
            pillbox_detected = True

        if label == 'Open':
            compartment_open_detected = True
            
            # Check if the open compartment matches the expected compartment
            if current_day_time and label not in current_day_time:
                wrong_compartment_opened = True

        # Debug statements to verify logic
        print(f"Label: {label}")
        print(f"Current Day and Time: {current_day_time}")
        print(f"Pillbox Detected: {pillbox_detected}")
        print(f"Compartment Open Detected: {compartment_open_detected}")
        print(f"Wrong Compartment Opened: {wrong_compartment_opened}")

    if pillbox_detected and compartment_open_detected and wrong_compartment_opened:
        send_buzzer_notification()

def send_buzzer_notification():
    global client_socket
    try:
        client_socket.sendall("Wrong pill detected".encode())
    except Exception as e:
        print(f"Failed to send buzzer notification: {e}")

# AI part
def process_frame(frame):
    # Perform inference with the YOLOv8 model
    try:
        results = model(frame)
    except Exception as e:
        print(f"Error performing inference: {e}")
        return frame

    # Annotate the frame with detection results
    draw_bounding_boxes(frame, results)

    return frame

# Tkinter GUI part
def update_frame():
    ret, frame = cap.read()
    if ret:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert colorspace from BGR to RGB
        frame = process_frame(frame) #Calls process_frame(frame) to perform object detection and annotation on the frame.
        img = Image.fromarray(frame)  #Converts the processed frame (which is a numpy array) to a PIL Image object using Image.fromarray.
        imgtk = ImageTk.PhotoImage(image=img) #Converts the PIL Image to a format suitable for displaying in a Tkinter GUI using ImageTk.PhotoImage.
        label_img.imgtk = imgtk #Updates the image displayed in the Tkinter Label widget (label_img) with the new frame by setting its image attribute to imgtk.
        label_img.configure(image=imgtk)
    else:
        print("Error reading frame from webcam")
    label_img.after(10, update_frame)  # Schedule the update after 10 milliseconds

# Initialize the webcam
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Setup Tkinter window
window = tk.Tk()
window.title("Pill Detection")
window.geometry("800x600")

# Create a label to display the webcam feed
label_img = tk.Label(window)
label_img.pack()

# Start the socket client
setup_socket_client()

# Request the initial date and time
request_date_time()

# Start the webcam feed update
update_frame()

# Start the Tkinter event loop
window.mainloop()

# Release the webcam
cap.release()


[ WARN:0@25.084] global /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_f6tvh9615u/croot/opencv-suite_1691620375715/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


Connected to server
Received from server: Wed-PM
Received from server: Wed-PM

0: 288x480 1 FRI-am, 1 FRI-pm, 1 MON-am, 1 MON-pm, 2 SAT-ams, 1 SAT-pm, 1 SUN-am, 1 SUN-pm, 1 THU-am, 1 THU-pm, 1 TUE-am, 1 TUE-pm, 1 WED-am, 1 WED-pm, 69.7ms
Speed: 2.0ms preprocess, 69.7ms inference, 1.5ms postprocess per image at shape (1, 3, 288, 480)
Label: WED-pm
Current Day and Time: Wed-PM
Pillbox Detected: False
Compartment Open Detected: False
Wrong Compartment Opened: False
Label: WED-am
Current Day and Time: Wed-PM
Pillbox Detected: False
Compartment Open Detected: False
Wrong Compartment Opened: False
Label: THU-am
Current Day and Time: Wed-PM
Pillbox Detected: False
Compartment Open Detected: False
Wrong Compartment Opened: False
Label: THU-pm
Current Day and Time: Wed-PM
Pillbox Detected: False
Compartment Open Detected: False
Wrong Compartment Opened: False
Label: SAT-pm
Current Day and Time: Wed-PM
Pillbox Detected: False
Compartment Open Detected: False
Wrong Compartment Opened: False
Label

[W NNPACK.cpp:53] Could not initialize NNPACK! Reason: Unsupported hardware.


TclError: image "pyimage1" doesn't exist

: 